In [2]:

# Simple R-tree Implementation 


class Rect:
    """Minimum Bounding Rectangle (MBR)"""
    def __init__(self, xmin, ymin, xmax, ymax):
        self.xmin, self.ymin, self.xmax, self.ymax = xmin, ymin, xmax, ymax

    def overlaps(self, other):
        """Check if two rectangles overlap"""
        return not (self.xmax < other.xmin or self.xmin > other.xmax or
                    self.ymax < other.ymin or self.ymin > other.ymax)

    def combine(self, other):
        """Return the smallest rectangle covering both"""
        return Rect(
            xmin=min(self.xmin, other.xmin),
            ymin=min(self.ymin, other.ymin),
            xmax=max(self.xmax, other.xmax),
            ymax=max(self.ymax, other.ymax)
        )

    def area(self):
        """Compute area of the rectangle"""
        return (self.xmax - self.xmin) * (self.ymax - self.ymin)

    def __repr__(self):
        return f"[({self.xmin},{self.ymin})-({self.xmax},{self.ymax})]"


class Entry:
    """One entry inside a node (MBR + child node or object ID)"""
    def __init__(self, mbr, child=None, obj_id=None):
        self.mbr = mbr
        self.child = child
        self.obj_id = obj_id


class Node:
    """R-tree node"""
    M = 4  # Max entries per node (kept small for demonstration)

    def __init__(self, is_leaf=True):
        self.is_leaf = is_leaf
        self.entries = []

    def mbr(self):
        """Return combined MBR of all entries"""
        if not self.entries:
            return None
        m = self.entries[0].mbr
        for e in self.entries[1:]:
            m = m.combine(e.mbr)
        return m


class RTree:
    """Main R-tree structure"""
    def __init__(self):
        self.root = Node(is_leaf=True)

    def insert(self, rect, obj_id):
        """Insert an object (simplified: no split)"""
        node = self.root
        node.entries.append(Entry(rect, child=None, obj_id=obj_id))
        if len(node.entries) > Node.M:
            print("⚠️ Node overflow (split logic omitted for simplicity)")

    def search(self, query):
        """Range query: find all objects intersecting with query"""
        results = []
        self._search_node(self.root, query, results)
        return results

    def _search_node(self, node, query, results):
        for e in node.entries:
            if e.mbr.overlaps(query):
                if node.is_leaf:
                    results.append(e.obj_id)
                else:
                    self._search_node(e.child, query, results)



#  Example: Insert and Query


# Create an R-tree
tree = RTree()

# Insert 3 rectangles (like buildings on a map)
tree.insert(Rect(1, 1, 3, 3), obj_id=101)  # Building A
tree.insert(Rect(5, 2, 7, 4), obj_id=102)  # Building B
tree.insert(Rect(2, 5, 4, 7), obj_id=103)  # Building C

# Query rectangle
query = Rect(2, 2, 6, 6)
result = tree.search(query)

# Display result
print(f"Query area: {query}")
print("Objects overlapping with query:", result)


Query area: [(2,2)-(6,6)]
Objects overlapping with query: [101, 102, 103]
